## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Polyarnyy,69.20,33.45,19.40,92,0,4.47,RU,clear sky
1,1,Passagem Franca,-6.18,-43.78,90.48,48,100,2.80,BR,overcast clouds
2,2,Qaanaaq,77.48,-69.36,12.99,87,100,7.18,GL,overcast clouds
3,3,Cape Town,-33.93,18.42,71.60,46,0,25.28,ZA,clear sky
4,4,Akdepe,42.06,59.38,57.20,24,0,9.17,TM,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 99


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
1,1,Passagem Franca,-6.18,-43.78,90.48,48,100,2.80,BR,overcast clouds
3,3,Cape Town,-33.93,18.42,71.60,46,0,25.28,ZA,clear sky
12,12,East London,-33.02,27.91,66.20,77,40,9.17,ZA,scattered clouds
13,13,Dingle,11.00,122.67,76.75,86,100,5.77,PH,overcast clouds
19,19,Rikitea,-23.12,-134.97,74.82,70,18,6.76,PF,few clouds
23,23,Banes,20.96,-75.72,80.60,83,75,14.99,CU,broken clouds
26,26,Bonthe,7.53,-12.51,86.38,64,42,5.59,SL,light rain
27,27,Cap Malheureux,-19.98,57.61,70.00,67,0,5.01,MU,clear sky
29,29,Gat,31.61,34.76,79.00,100,32,1.99,IL,scattered clouds
33,33,Ixtapa,20.70,-105.20,77.00,88,5,4.70,MX,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID        853
City           853
Lat            853
Lng            853
Max Temp       853
Humidity       853
Cloudiness     853
Wind Speed     853
Country        848
Description    853
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
1,1,Passagem Franca,-6.18,-43.78,90.48,48,100,2.80,BR,overcast clouds
3,3,Cape Town,-33.93,18.42,71.60,46,0,25.28,ZA,clear sky
12,12,East London,-33.02,27.91,66.20,77,40,9.17,ZA,scattered clouds
13,13,Dingle,11.00,122.67,76.75,86,100,5.77,PH,overcast clouds
19,19,Rikitea,-23.12,-134.97,74.82,70,18,6.76,PF,few clouds
...,...,...,...,...,...,...,...,...,...,...
1815,1815,Kahului,20.89,-156.47,78.80,65,20,5.82,US,few clouds
1816,1816,Souillac,-20.52,57.52,71.60,68,75,14.99,MU,shower rain
1817,1817,Hilo,19.73,-155.09,75.20,78,90,2.89,US,overcast clouds
1819,1819,Berekua,15.23,-61.32,86.00,58,40,3.36,DM,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Passagem Franca,BR,90.48,overcast clouds,-6.18,-43.78,
3,Cape Town,ZA,71.60,clear sky,-33.93,18.42,
12,East London,ZA,66.20,scattered clouds,-33.02,27.91,
13,Dingle,PH,76.75,overcast clouds,11.00,122.67,
19,Rikitea,PF,74.82,few clouds,-23.12,-134.97,
23,Banes,CU,80.60,broken clouds,20.96,-75.72,
26,Bonthe,SL,86.38,light rain,7.53,-12.51,
27,Cap Malheureux,MU,70.00,clear sky,-19.98,57.61,
29,Gat,IL,79.00,scattered clouds,31.61,34.76,
33,Ixtapa,MX,77.00,clear sky,20.70,-105.20,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame. 
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
   
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


Figure(layout=FigureLayout(height='420px'))

In [9]:
# hotel_df check

hotel_clean_df = hotel_df.loc[(hotel_df["Hotel Name"] != "NaN")]
hotel_clean_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Passagem Franca,BR,90.48,overcast clouds,-6.18,-43.78,Hotel Brasil
3,Cape Town,ZA,71.60,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town
12,East London,ZA,66.20,scattered clouds,-33.02,27.91,Tu Casa
13,Dingle,PH,76.75,overcast clouds,11.00,122.67,Camp Pasica
19,Rikitea,PF,74.82,few clouds,-23.12,-134.97,Pension Maro'i
...,...,...,...,...,...,...,...
1815,Kahului,US,78.80,few clouds,20.89,-156.47,Maui Seaside Hotel
1816,Souillac,MU,71.60,shower rain,-20.52,57.52,Bils' Villa
1817,Hilo,US,75.20,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
1819,Berekua,DM,86.00,scattered clouds,15.23,-61.32,CarRod's Cottages


In [10]:
hotel_clean_df.count()

City           848
Country        848
Max Temp       848
Description    848
Lat            848
Lng            848
Hotel Name     848
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.

hotel_clean_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Passagem Franca,BR,90.48,overcast clouds,-6.18,-43.78,Hotel Brasil
3,Cape Town,ZA,71.60,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town
12,East London,ZA,66.20,scattered clouds,-33.02,27.91,Tu Casa
13,Dingle,PH,76.75,overcast clouds,11.00,122.67,Camp Pasica
19,Rikitea,PF,74.82,few clouds,-23.12,-134.97,Pension Maro'i
...,...,...,...,...,...,...,...
1815,Kahului,US,78.80,few clouds,20.89,-156.47,Maui Seaside Hotel
1816,Souillac,MU,71.60,shower rain,-20.52,57.52,Bils' Villa
1817,Hilo,US,75.20,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
1819,Berekua,DM,86.00,scattered clouds,15.23,-61.32,CarRod's Cottages


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")



In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#The end